## Import bibliotek

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tslearn.utils import to_time_series_dataset
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sktime.clustering.k_means import TimeSeriesKMeans as TSKM
from sktime.distances import dtw_distance, pairwise_distance, lcss_distance, wddtw_distance


### Definicja zmiennych

In [3]:
params = [ "pcm_LOGenergy_sma",
    "pcm_zcr_sma" ,
    "voiceprob_sma" ,
    "f0_sma" ,
    "f0env_sma",
    "pcm_fftMag_fband0-250_sma" ,
    "pcm_fftMag_fband0-650_sma",
    "pcm_fftMag_spectralRollOff25_0_sma" ,
    "pcm_fftMag_spectralRollOff50_0_sma",
    "pcm_fftMag_spectralRollOff75_0_sma",
    "pcm_fftMag_spectralRollOff90_0_sma", 
    "pcm_fftmag_spectralflux_sma",
    "pcm_fftmag_spectralcentroid_sma" ,
    "pcm_fftmag_spectralmaxpos_sma" ,
    "pcm_fftmag_spectralminpos_sma" ,
    "f0final_sma" ,
    "voicingfinalunclipped_sma",
    "jitterlocal_sma",
    "jitterddp_sma",
    "shimmerlocal_sma",
    "loghnr_sma",
    "audspec_lengthl1norm_sma" ,
    "audspecrasta_lengthl1norm_sma" ,
    "pcm_rmsenergy_sma",
    "audSpec_Rfilt_sma_compare_0_",
    "audSpec_Rfilt_sma_compare_1_" ,
    "audSpec_Rfilt_sma_compare_2_" ,
    "audSpec_Rfilt_sma_compare_3_" ,
    "audSpec_Rfilt_sma_compare_4_",
    "audSpec_Rfilt_sma_compare_5_",
    "audSpec_Rfilt_sma_compare_6_" ,
    "audSpec_Rfilt_sma_compare_7_",
    "audSpec_Rfilt_sma_compare_8_",
    "audSpec_Rfilt_sma_compare_9_",
    "audSpec_Rfilt_sma_compare_10_",
    "audSpec_Rfilt_sma_compare_11_",
    "audSpec_Rfilt_sma_compare_12_",
    "audSpec_Rfilt_sma_compare_13_",
    "audSpec_Rfilt_sma_compare_14_",
    "audSpec_Rfilt_sma_compare_15_" ,
    "audSpec_Rfilt_sma_compare_16_",
    "audSpec_Rfilt_sma_compare_17_",
    "audSpec_Rfilt_sma_compare_18_" ,
    "audSpec_Rfilt_sma_compare_19_" ,
    "audSpec_Rfilt_sma_compare_20_" ,
    "audSpec_Rfilt_sma_compare_21_" ,
    "audSpec_Rfilt_sma_compare_22_" ,
    "audSpec_Rfilt_sma_compare_23_" ,
    "audSpec_Rfilt_sma_compare_24_" ,
    "audSpec_Rfilt_sma_compare_25_",
    "pcm_fftMag_fband250-650_sma_compare" ,
    "pcm_fftMag_fband1000-4000_sma_compare",
    "pcm_fftmag_spectralentropy_sma_compare",
    "pcm_fftmag_spectralvariance_sma_compare" ,
    "pcm_fftmag_spectralskewness_sma_compare",
    "pcm_fftmag_spectralkurtosis_sma_compare" ,
    "pcm_fftmag_psysharpness_sma_compare" ,
    "pcm_fftmag_spectralharmonicity_sma_compare",
    "loudness_sma3",
    "alpharatio_sma3",
    "hammarbergindex_sma3" ,
    "slope0-500_sma3" ,
    "slope500-1500_sma3",
    "F0semitoneFrom27_5Hz_sma3nz",
    "logRelF0-H1-H2_sma3nz",
    "logRelF0-H1-A3_sma3nz" ,
    "f1frequency_sma3nz" ,
    "f1bandwidth_sma3nz" ,
    "f1amplitudelogrelf0_sma3nz" ,
    "f2frequency_sma3nz" ,
    "f2amplitudelogrelf0_sma3nz" ,
    "f3frequency_sma3nz" ,
    "f3amplitudelogrelf0_sma3nz",
    "pcm_fftMag_mfcc_0_" ,
    "pcm_fftMag_mfcc_1_" ,
    "pcm_fftMag_mfcc_2_" ,
    "pcm_fftMag_mfcc_3_",
    "pcm_fftMag_mfcc_4_" ,
    "pcm_fftMag_mfcc_5_" ,
    "pcm_fftMag_mfcc_6_",
    "pcm_fftMag_mfcc_7_",
    "pcm_fftMag_mfcc_8_",
    "pcm_fftMag_mfcc_9_",
    "pcm_fftMag_mfcc_10_",
    "pcm_fftMag_mfcc_11_",
    "pcm_fftMag_mfcc_12_"
  ]
colnames = ['day','hamd_ymrs','patient_id', 'dw_mobilerecording_id', 'frame_nr'] + params

# PARAMETRY

In [62]:
frame_no = 1000
cluster_no = 2
patient_id = 2582

### Pobranie danych dla 1 pacjenta

In [5]:
#TODO: DO ZASTANOWIENIA SIĘ ILE WYBIERAMY PIERWSZYCH RAMEK!!!

In [6]:
danonek = pd.read_csv("C:\\Users\\user\\Documents\\danonki\\2582_data.csv" ) 
#Wybieram pierwsze 1000 ramek!
danonek = danonek.loc[ (danonek.frame_nr <= frame_no) & (danonek.chunk_number == 0), :]
danonek_bak = danonek

In [7]:
mr_v = pd.read_csv("C:\\Users\\user\\Documents\\recordings_visits.csv") 
rec = pd.read_csv("C:\\Users\\user\\Documents\\danonki\\mobilerecordings.csv")

In [8]:
# WYbieram dane od 2018 roku
rec = rec.loc[rec.create_date >= '2018-01-01',:]

In [9]:
#połączenie czystego nagrania z datą
danonek = pd.merge(danonek, mr_v,  how='left', left_on=['patient_id','dwh_mobilerecording_id'],
                    right_on = ['patient_id','dwh_mobilerecording_id'])

In [10]:
test = [danonek_bak.columns]

## Selekcja zmiennych

In [11]:
ficzery = [
 #'f0_sma',
# 'pcm_fftMag_fband0-250_sma',
# 'pcm_fftMag_fband0-650_sma',
 'f0final_sma',
# 'slope0-500_sma3',
# 'slope500-1500_sma3',
 'f1frequency_sma3nz',
 'f2frequency_sma3nz',
 'f3frequency_sma3nz',
 'pcm_fftMag_mfcc_5_']

In [12]:
description =  ficzery + [ 'dwh_mobilerecording_id', 'frame_nr', 'create_date','hamd_ymrs', 'visit_number', 'hamd_suma', 'yms_suma', 'visit_day']

In [13]:
danonek = danonek[description]

## Wyznaczenie znanych stanów:

In [14]:
#visits_no = danonek.visit_number.dropna().unique()

### Podział na dane olabelowane i nieolabelowane

In [15]:
train_sets = [] #zawiera liste tabel oetykietowanych
train_sets = [group for _, group in danonek.groupby('visit_number')]

In [16]:
test_set = danonek.loc[danonek.hamd_ymrs.isna(),: ] 

## Czyszczenie danych

In [17]:
all_set = danonek.drop_duplicates().reset_index(drop=True).sort_values(['create_date','dwh_mobilerecording_id', 'frame_nr'])

In [18]:
# Usuwam nagrania które nie mają 1000 ramek!!
count = all_set['dwh_mobilerecording_id'].value_counts() 
val_to_remove = count[count < frame_no].index.tolist()
all_set = all_set[~all_set['dwh_mobilerecording_id'].isin(val_to_remove)]
del(count, val_to_remove)

In [19]:
train_sets = [df.drop_duplicates().reset_index(drop=True).sort_values(['dwh_mobilerecording_id', 'frame_nr']) 
    for df in train_sets]

In [20]:
# Usuwam nagrania które nie mają 1000 ramek!!

for i in range(len(train_sets)):
    df = train_sets[i]
    count = df['dwh_mobilerecording_id'].value_counts() 
    val_to_remove = count[count < frame_no].index.tolist()
    train_sets[i] = df[~df['dwh_mobilerecording_id'].isin(val_to_remove)]


In [21]:
test_set = test_set.drop_duplicates().reset_index(drop=True).sort_values(['dwh_mobilerecording_id', 'frame_nr'])

In [22]:
# Usuwam nagrania które nie mają 1000 ramek!!
count = test_set['dwh_mobilerecording_id'].value_counts() 
val_to_remove = count[count < frame_no].index.tolist()
test_set = test_set[~test_set['dwh_mobilerecording_id'].isin(val_to_remove)]
del(count, val_to_remove)

## TRAIN

In [61]:
len(ts_ficzer)

13

In [72]:
df_output_bak = df_output

In [82]:
#dla każdego dnia badania robie clustering na wszystkich parametrach!!!!
# a później dopiero porównuje czy to był dzień etykietowany czy nie!!!!

days = list(all_set.create_date.unique())

km = TSKM(n_clusters=cluster_no,
                  metric = 'dtw') # co to:  ‘ddtw’, ‘wdtw’, ‘wddtw’ <- do sprawdzenia


rejected_rec = []
df_output = pd.DataFrame(columns=['create_date','patient','feature','yms_suma','hamd_suma','hamd_ymrs',
                                  'visit_number','visit_day', 'dtw', 'wddtw', 
                                  'cl0_ile', 'cl1_ile', 'ile_nagran'])

for d in days:
    print(d)
    for param in ficzery:
        ts_list = all_set.loc[all_set.create_date == d,:].groupby('dwh_mobilerecording_id')[param].apply(list).reset_index(drop=True)
        if(len(ts_list)<2): #jeśli mieliśmy tylko jedno nagranie w 1 dniu to odrzucamy
            rejected_rec.append(d)
            continue
        ts_ficzer = to_time_series_dataset(ts_list)
        ts_ficzer = TimeSeriesScalerMinMax().fit_transform(ts_ficzer)
        km.fit(ts_ficzer)
        #clusters_centers = km.cluster_centers_
        #clusters_labels = km.labels_
        dtw = dtw_distance(km.cluster_centers_[0].ravel(), km.cluster_centers_[1].ravel()  )
        wddtw = wddtw_distance(km.cluster_centers_[0].ravel(), km.cluster_centers_[1].ravel()  )

        new_row = pd.DataFrame({'create_date' : [d],
                                'patient' : [patient_id],
                                'feature' : [param],
                                'yms_suma' : [all_set.loc[all_set.create_date == d,'yms_suma'].unique()[0]],
                                'hamd_suma' :  [all_set.loc[all_set.create_date == d,'hamd_suma'].unique()[0]],
                                'hamd_ymrs' :  [all_set.loc[all_set.create_date == d,'hamd_ymrs'].unique()[0]], 
                                'visit_number' :  [all_set.loc[all_set.create_date == d,'visit_number'].unique()[0]],
                                'visit_day' :  [all_set.loc[all_set.create_date == d,'visit_day'].unique()[0]],
                            'dtw' : [ dtw],
                            'wddtw' : [wddtw],
                            'cl0_ile' :   [np.sum(km.labels_ == 0)],
                            'cl1_ile'  : [np.sum(km.labels_ == 1)],
                            'ile_nagran' : [len(ts_ficzer)]
                            })
        df_output  = pd.concat([df_output , new_row], ignore_index=True)

    
 


#30 min!
##############################################################################################################
 

2018-01-06
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-24
2018-03-25
2018-03-26
2018-03-28
2018-03-29
2018-03-30
2018-03-31
2018-04-01
2018-04-03
2018-04-05
2018-04-06
2018-04-07
2018-04-08
2018-04-09
2018-04-10
2018-04-12
2018-04-13
2018-04-14
2018-04-15
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-21
2018-04-22
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-28
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-04
2018-05-05
2018-05-07
2018-05-08
2018-05-09
2018-05-10
2018-05-11
2018-05-12
2018-05-13
2018-06-03
2018-06-04
2018-06-05
2018-06-06
2018-06-07
2018-06-08
2018-06-09
2018-06-10
2018-06-11
2018-06-12
2018-06-13
2018-06-14
2018-06-15
2018-06-16
2018-06-17
2018-06-18
2018-06-19
2018-06-20


## Wykres dla zbioru treningowego

In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd

# def plot_lines(data_list):
#     # Create a new figure and axis
#     fig, ax = plt.subplots()

#     # Plot lines for each element of the list
#     for i in range(5):#range(len(data_list)):
#         series = data_list[i]
#         x = series.index
#         y = series.f1frequency_sma3nz
#         ax.plot(x, y, label= series.dwh_mobilerecording_id.unique())

#     # Set labels and title
#     ax.set_xlabel('X-axis')
#     ax.set_ylabel('Y-axis')
#     ax.set_title('Lines for List of Series')

#     # Add a legend
#     ax.legend(alignment = 'right')

#     # Display the plot
#     plt.show()


# plot_lines(train_data)
# plot_lines(test_data)

## Wykres dla zbioru testowego

In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd

# def plot_lines(data_list):
#     # Create a new figure and axis
#     fig, ax = plt.subplots()

#     # Plot lines for each element of the list
#     for i in range(10):#range(len(data_list)):
#         series = data_list[i]
#         x = series.index
#         y = series.f1frequency_sma3nz
#         ax.plot(x, y, label= series.dwh_mobilerecording_id.unique())

#     # Set labels and title
#     ax.set_xlabel('X-axis')
#     ax.set_ylabel('Y-axis')
#     ax.set_title('Lines for List of Series')

#     # Add a legend
#     ax.legend(alignment = 'right')

#     # Display the plot
#     plt.show()


# plot_lines(dane_danonek)

In [ ]:
#dan_p['create_date'] = pd.to_datetime(dan_p['create_date_x'], format='%Y-%m-%d')

# KMEANS

In [ ]:
 
# for yi in range(liczba_klastrow):
#     plt.subplot(liczba_klastrow, liczba_klastrow)
#     for rec in x_train_tes :
#         plt.plot(rec.ravel(), "k-", alpha=.2)
#     plt.plot(model3.cluster_centers_[yi].ravel(), "r-")
#     plt.xlim(0, sz)
#     plt.ylim(-4, 4)
#     plt.text(0.55, 0.85,'Cluster %d' % (yi + 1))


i=0
l = 0
for rec in x_train_ts :
    if(labelki_klastrow[i] == 0):
        plt.plot(rec.ravel(), color='green')
        l=l+1
    i=i+1

plt.plot(centrum_klastrow[0].ravel(),color='red'  )
plt.plot(centrum_klastrow[1].ravel(),color='blue'  )
plt.title("Cluster 1")
print(i)
print(l)

In [ ]:
j=0
k=0
for rec in x_train_ts :
    if(labelki_klastrow[j] == 1):
        plt.plot(rec.ravel(), color='green')
        k=k+1
    j=j+1
plt.title("Cluster 2")
plt.plot(centrum_klastrow[1].ravel(),color='red'  )
print(j)
print(k)

In [ ]:
# dni_no_label = danonek_nolabel_param1.create_date.unique()
# for day in dni_no_label:
#     day_one = danonek_nolabel_param1.loc[danonek_nolabel_param1.create_date == day, :]
#     break

## Podejście per dzień

In [ ]:
danonek_label_all = danonek.loc[ ~danonek.hamd_ymrs.isnull(),: ]
danonek_label_param1_all = danonek_label_all[ ['f1frequency_sma3nz', 'dwh_mobilerecording_id', 'frame_nr', 'create_date','hamd_ymrs']].drop_duplicates().reset_index(drop=True).sort_values(['dwh_mobilerecording_id', 'frame_nr'])
#danonek_label_param1_all

recordings_label_all = danonek_label_param1_all.dwh_mobilerecording_id.unique()
recordings_label_all
train_data100x = []

for i in recordings_label_all:
        n_frames = len(danonek_label_param1_all.loc[danonek_label_param1_all.dwh_mobilerecording_id == i, ['f1frequency_sma3nz']].reset_index(drop=True))
        if(n_frames <= 100): #jeśli nagranie ma mniej niż 100 ramek to nie bierzemy go pod uwagę
                print(i)
                print(n_frames)
                continue
        train_data100x.append(danonek_label_param1_all.loc[danonek_label_param1_all.dwh_mobilerecording_id == i,
         ['dwh_mobilerecording_id','f1frequency_sma3nz','create_date', 'hamd_ymrs']].reset_index(drop=True).head(100)) #wybieram pozostałe informacje dla każdego nagrania

#wybieram pierwsze 100 ramek z każdego nagrania
#train_data100 = [df.head(100) for df in train_data]

In [ ]:
len(train_data100x)#[12]
train_data100x[112]

In [ ]:
liczba_klastrow = 2
km = TSKM(n_clusters=liczba_klastrow,
                  metric = 'dtw')

combined_df_train = pd.concat(train_data100x)
# Group the combined DataFrame by the 'date' column
grouped_train = combined_df_train.groupby('create_date')

#train_frames = []
#train_x2 = []
centrum_klastrow_list_train = []
recs_in_day_train = []

df_output_train = pd.DataFrame(columns=['create_date', 'dtw', 'wddtw', 
                                  'cl0_ile', 'cl1_ile', 'ile_nagran'])

# Iterate over the groups
for date, group in grouped_train:
    # Create a new DataFrame for each unique date
    unique_date_df_train = pd.DataFrame(group)

    grouped_by_id  = unique_date_df_train.groupby('dwh_mobilerecording_id')
    print(date)
    lista_dzien = []
    for recording_id, sub_group in grouped_by_id:
        frequency_values = sub_group['f1frequency_sma3nz'].tolist()
        lista_dzien.append(frequency_values)
        ficzur = to_time_series_dataset( lista_dzien )
        
    if (len(ficzur) < 2 ): #jeśli jest tylko 1 nagranie
        continue
    print(len(ficzur))
    ficzur = TimeSeriesScalerMinMax().fit_transform(ficzur )
    km.fit(ficzur)
    centrum_klastrow_list_train.append(km.cluster_centers_)
    
    new_row = pd.DataFrame({'create_date' : [date],
                            'dtw' : [dtw_distance(km.cluster_centers_[0].ravel(), km.cluster_centers_[1].ravel())],
                            'wddtw' : [wddtw_distance(km.cluster_centers_[0].ravel(), km.cluster_centers_[1].ravel())],
                            'cl0_ile' :   [np.sum(km.labels_ == 0)],
                            'cl1_ile'  : [np.sum(km.labels_ == 1)],
                            'ile_nagran' : [len(ficzur)]
                            })
    df_output_train = pd.concat([df_output_train, new_row], ignore_index=True)



# TUTAJ SKOŃCZYŁAM <3 

In [ ]:
vvv_basic = mr_v.loc[(~mr_v.hamd_ymrs.isnull() ) & (mr_v.patient_id == 2582) ,['create_date','hamd_ymrs','visit_day','patient_id']].drop_duplicates()

In [ ]:
#df_output_train.join(mr_v.loc[~mr_v.hamd_ymrs.isnull(),['create_date','hamd_ymrs']].drop_duplicates(),how='left',on='create_date')
vvv_basic = mr_v.loc[(~mr_v.hamd_ymrs.isnull() ) & (mr_v.patient_id == 2582) ,['create_date','hamd_ymrs','visit_day','patient_id']].drop_duplicates()
idx = vvv_basic.groupby('create_date')['visit_day'].apply(lambda x: x.abs().idxmin())

# Use the index to filter the original DataFrame
vvv = vvv_basic.loc[idx]
vvv
ramka_trainn = pd.merge(df_output_train, vvv , how= 'inner' , on = 'create_date')
ramka_trainn.loc[ramka_trainn.visit_day.isin([0,1,-1,-2,-3]),:]

In [ ]:
np.std([8.68,2.69,7.14,2.73])
np.std([1.78,1,1.37,2.03,1.1])


In [ ]:
mr_v.loc[~mr_v.hamd_ymrs.isnull(),['create_date','hamd_ymrs']].drop_duplicates()

## Porównywanie danych nieotykietowanych 

In [ ]:

recordings_nolabel = danonek_nolabel_param1.dwh_mobilerecording_id.unique()
test_x = []
test_y = []
test_data = []

for i in recordings_nolabel:
        n_frames = len(danonek_nolabel_param1.loc[danonek_nolabel_param1.dwh_mobilerecording_id == i, ['f1frequency_sma3nz']].reset_index(drop=True))
        if(n_frames < 100): #jeśli nagranie ma mniej niż 100 ramek to nie bierzemy go pod uwagę
                print(i)
                print(n_frames)
                continue
        test_x.append(danonek_nolabel_param1.loc[danonek_nolabel_param1.dwh_mobilerecording_id == i,
         ['f1frequency_sma3nz']].reset_index(drop=True).values[0:100]) #wybieram tylko wartości parametru 1
        test_y.append(danonek_nolabel_param1.loc[danonek_nolabel_param1.dwh_mobilerecording_id == i,
         'hamd_ymrs'].drop_duplicates().reset_index(drop=True)) #wybieram etykiete dla każdego nagrania 
        test_data.append(danonek_nolabel_param1.loc[danonek_nolabel_param1.dwh_mobilerecording_id == i,
         ['dwh_mobilerecording_id','f1frequency_sma3nz','create_date', 'hamd_ymrs']].reset_index(drop=True)) #wybieram pozostałe informacje dla każdego nagrania
        

from tslearn.utils import to_time_series_dataset

x_test_ts = to_time_series_dataset( test_x ) #lista 1447 nagrań po 100 wartości

y_test_ts =  test_y
x_test_ts = TimeSeriesScalerMinMax().fit_transform(x_test_ts )  


In [ ]:
#x_test_ts.shape

# Check for NaN values
nan_indices = np.isnan(x_test_ts)

# Get the indices of NaN values using np.where
nan_indices = np.where(nan_indices)

# Print the indices of NaN values
for i, j, k in zip(*nan_indices):
    print(f"NaN value found at index ({i}, {j}, {k})")

In [ ]:
liczba_klastrow = 2
km = TSKM(n_clusters=liczba_klastrow,
                  metric = 'dtw')  # co to:  ‘ddtw’, ‘wdtw’, ‘wddtw’ <- do sprawdzenia
km.fit(x_test_ts)  

In [ ]:
centrum_klastrow = km.cluster_centers_
labelki_klastrow = km.labels_
inertia = km.inertia_
it = km.n_iter_

In [ ]:
#wybieram pierwsze 100 ramek z każdego nagrania
test_data100 = [df.head(100) for df in test_data]

In [ ]:

liczba_klastrow = 2
km = TSKM(n_clusters=liczba_klastrow,
                  metric = 'dtw')

combined_df = pd.concat(test_data100)
# Group the combined DataFrame by the 'date' column
grouped = combined_df.groupby('create_date')

test_frames = []
test_x2 = []
centrum_klastrow_list = []
recs_in_day = []

df_output = pd.DataFrame(columns=['create_date', 'dtw', 'wddtw', 
                                  'cl0_ile', 'cl1_ile', 'ile_nagran'])

# Iterate over the groups
for date, group in grouped:
    # Create a new DataFrame for each unique date
    unique_date_df = pd.DataFrame(group)

    grouped_by_id  = unique_date_df.groupby('dwh_mobilerecording_id')
    print(date)
    lista_dzien = []
    for recording_id, sub_group in grouped_by_id:
        frequency_values = sub_group['f1frequency_sma3nz'].tolist()
        lista_dzien.append(frequency_values)
        ficzur = to_time_series_dataset( lista_dzien )
        
    if (len(ficzur) < 2 ): #jeśli jest tylko 1 nagranie
        continue
    print(len(ficzur))
    ficzur = TimeSeriesScalerMinMax().fit_transform(ficzur )
    km.fit(ficzur)
    centrum_klastrow_list.append(km.cluster_centers_)
    
    new_row = pd.DataFrame({'create_date' : [date],
                            'dtw' : [dtw_distance(km.cluster_centers_[0].ravel(), km.cluster_centers_[1].ravel())],
                            'wddtw' : [wddtw_distance(km.cluster_centers_[0].ravel(), km.cluster_centers_[1].ravel())],
                            'cl0_ile' :   [np.sum(km.labels_ == 0)],
                            'cl1_ile'  : [np.sum(km.labels_ == 1)],
                            'ile_nagran' : [len(ficzur)]
                            })
    df_output = pd.concat([df_output, new_row], ignore_index=True)

    #if date == '2018-03-22':
    #    break

# TUTAJ SKOŃCZYŁAM <3 

In [ ]:
df_output

In [ ]:
i=0
l = 0
for rec in x_test_ts :
    if(labelki_klastrow[i] == 0):
        plt.plot(rec.ravel(), color='green')
        l=l+1
    i=i+1

plt.plot(centrum_klastrow[0].ravel(),color='red'  )
plt.title("Cluster 1")
print(i)
print(l)

In [ ]:
j=0
k=0
for rec in x_test_ts :
    if(labelki_klastrow[j] == 1):
        plt.plot(rec.ravel(), color='green')
        k=k+1
    j=j+1
plt.title("Cluster 2")
plt.plot(centrum_klastrow[1].ravel(),color='red'  )
print(j)
print(k)

In [ ]:
import numpy as np
from sktime.distances import dtw_distance, pairwise_distance, lcss_distance, wddtw_distance

#wddtw_distance(centrum_klastrow[0].ravel(), centrum_klastrow[1].ravel()  )
dtw_distance(centrum_klastrow[0].ravel(), centrum_klastrow[1].ravel()  )
#wddtw_distance(x_train_ts[1:10,1], x_train_ts[51:60,1])